In [ ]:
import json
from arcgis.gis import GIS
from arcgis.features import FeatureLayer
from arcgis.features import FeatureLayerCollection
import pandas as pd

In [ ]:
# connect to ArcGIS Online with DNR credentials

with open(r'C:\projects\python\jupyter\secrets.json') as secrets:
    s = json.load(secrets)
    
    # load username and password from secrets.json into variables here
    DNR_username = s["dnr"]["username"]
    DNR_password = s["dnr"]["password"]

# connect to the GIS
gis = GIS(username=DNR_username, password=DNR_password)

In [ ]:
# URL of Employee Homes FeatureLayer
employees_url=r'https://services.arcgis.com/ZzrwjTRez6FJiOq4/arcgis/rest/services/COVID19_DNR_Resources_gdb/FeatureServer/0'

# the attribute table from this layer will be read into Pandas as a dataframe
# that dataframe will be exported as a CSV
# that CSV will be the source of the List in the dashboard

employees_flayer = FeatureLayer(url=employees_url)
emp_fset = employees_flayer.query()

# convert the DataSet to a Pandas dataframe
emp_pd = emp_fset.sdf

emp_pd.head(10)

In [ ]:
# Group By 'Division' and Sum 'Status' fields
emp_group = emp_pd.groupby(['Division', 'Status'])

# create a new dataframe from groups, using the size() aggregation function
output = pd.DataFrame(emp_group.size())

output

In [ ]:
# this dataframe is using a MultiIndex for the rows
print(output.index)


In [ ]:
# there is only one single column of actual data
print("shape:", output.shape)

# rename the count column
output.columns = ["Count"]

# display the dataframe
output

In [ ]:
# reformat dataframe for the pivot here

# reset the index to remove the multi-index
pivot_output = output.reset_index()

pivot_output

In [ ]:
# do the pivot
pivot_div = pivot_output.pivot(index='Division', columns='Status', values='Count')

# change the NaN values to 0
pivot_div = pivot_div.fillna(0)

# cast the series to integers (to remove decimals appearing in the output csv)
pivot_div.astype({'Active': 'int32', 'Annual': 'int32', 'Leave Without Pay': 'int32', 'Sick': 'int32'})

In [ ]:
# save the dataframe to a CSV

csv_filename = r'c:\test\division_status_summary_pivot.csv'
pivot_div.to_csv(csv_filename)

# FIX:
# Pandas is still exporting integers as floats, with a .0 after each digit
# this may need to be fixed by opening the CSV in Python and resaving it
# with the proper integer formatting

In [ ]:
# publish the CSV to ArcGIS Online as a hosted feature layer
# add it to a web map
# add that web map to a Dashboard, in order to access the table

# upload (overwrite) the existing CSV on ArcGIS Online
# see article: https://developers.arcgis.com/python/sample-notebooks/overwriting-feature-layers/

# ArcGIS Online ItemID of hosted summary CSV table
# summarycsv_id = '11e7b670a2fc4346a5f4d687aeb73345' # old
summarycsv_id = 'ca018835d19a4f9fb2cbdf4ee2428162'

# get the CSV Table Item from the gis object
sumcsv_item = gis.content.get(summarycsv_id)

# get a FeatureLayerCollection to overwrite the table
csv_flayer_collection = FeatureLayerCollection.fromitem(sumcsv_item)
csv_flayer_collection.manager.overwrite(csv_filename)